<a href="https://colab.research.google.com/github/EdH66/CMAP-Analysis/blob/main/MPO_automatic_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multiparameter optimisation (MPO) calculator

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
# Read the CSV file
import pandas as pd

path_chemical_MPO = '/content/drive/My Drive/Colab Notebooks/CMAP Drug Prediction/MPO_autocalc.csv'
chemical_MPO = pd.read_csv(path_chemical_MPO)

print(chemical_MPO.head())



         Drug  MPO     mw  logp   TSA  logd  HBD  basic pka
0  Alprazolam  6.0  309.0   2.6  43.0   1.9  0.0        2.4
1  Aniracetam  6.0  219.0   0.7  47.0   1.3  0.0        1.0
2    Caffeine  6.0  194.0   0.0  62.0  -0.6  0.0        1.0
3    Zaleplon  6.0  305.0   1.4  74.0   1.4  0.0        1.0
4  Flumazenil  5.9  303.0   1.3  64.0   2.2  0.0        1.0


## LogP function

In [63]:
def normalize_logP(value):
    if value < 3:
        return 1
    elif 3 <= value < 5:
        return 1 - (value - 3) / 2  # Ramp from 1 to 0 over the range 3 to 5
    else:
        return 0

# Assuming df is your DataFrame and 'LogP' is the column name
chemical_MPO['Normalized_LogP'] = chemical_MPO['logp'].apply(normalize_logP)


## LogD function

In [64]:
def normalize_logD(value):
    if value < 2:
        return 1
    elif 2 <= value < 4:
        return 1 - (value - 2) / 2  # Ramp from 1 to 0 over the range 2 to 4
    else:
        return 0

chemical_MPO['Normalized_LogD'] = chemical_MPO['logd'].apply(normalize_logD)


## Molecular weight function

In [65]:
def normalize_mol_weight(value):
    if value < 360:
        return 1
    elif 360 <= value < 500:
        return 1 - (value - 360) / 140  # Ramp from 1 to 0 over the range 360 to 500
    else:
        return 0

chemical_MPO['Normalized_Mol_Weight'] = chemical_MPO['mw'].apply(normalize_mol_weight)


#Topological Surface Area function

In [66]:
def normalize_TPSA(value):
    if value < 20:
        return 0
    elif 20 <= value < 40:
        return (value - 20) / 20  # Ramp from 0 to 1 over the range 20 to 40
    elif 40 <= value <= 90:
        return 1
    elif 90 < value <= 120:
        return 1 - (value - 90) / 30  # Ramp from 1 to 0 over the range 90 to 120
    else:
        return 0

chemical_MPO['Normalized_TPSA'] = chemical_MPO['TSA'].apply(normalize_TPSA)


#h-bond donor function

In [67]:
def normalize_hbond_donor(value):
    if value == 0:
        return 1
    elif value == 1:
        return 0.75
    elif value == 2:
        return 0.5
    elif value == 3:
        return 0.25
    else:
        return 0

chemical_MPO['Normalized_Hbond_Donor'] = chemical_MPO['HBD'].apply(normalize_hbond_donor)


# basic pKA function

In [68]:
def normalize_pKa(value):
    if value < 8:
        return 1
    elif 8 <= value < 10:
        # Linear decrease from 1 to 0 over the range of 8 to 10
        return 1 - (value - 8) / 2
    else:
        return 0

chemical_MPO['Normalized_Most_Basic_pKa'] = chemical_MPO['basic pka'].apply(normalize_pKa)



#Calculate MPO Score

In [72]:
import pandas as pd

# Find all columns that contain 'Normalized' in their title
normalized_cols = [col for col in chemical_MPO.columns if 'Normalized' in col]

# Create 'Calculated_MPO' column by summing normalized values
chemical_MPO['Calculated_MPO'] = chemical_MPO[normalized_cols].sum(axis=1)

# Move 'Calculated_MPO' to the beginning
columns = ['Calculated_MPO'] + [col for col in chemical_MPO.columns if col != 'Calculated_MPO']

# Reorder the DataFrame columns
chemical_MPO = chemical_MPO[columns]

# Add 'Good_Flag' column
chemical_MPO['Good_Flag'] = chemical_MPO.apply(lambda x: 'Good' if abs(x['Calculated_MPO'] - x['MPO']) <= 0.25 else '', axis=1)

# Display the DataFrame
print(chemical_MPO[['Calculated_MPO', 'MPO']].head())

# Display the DataFrame
# Print the DataFrame
print(chemical_MPO)


   Calculated_MPO  MPO
0             6.0  6.0
1             6.0  6.0
2             6.0  6.0
3             6.0  6.0
4             5.9  5.9
     Calculated_MPO         Drug  MPO      mw  logp   TSA  logd  HBD  \
0               6.0   Alprazolam  6.0  309.00  2.60  43.0  1.90  0.0   
1               6.0   Aniracetam  6.0  219.00  0.70  47.0  1.30  0.0   
2               6.0     Caffeine  6.0  194.00  0.00  62.0 -0.60  0.0   
3               6.0     Zaleplon  6.0  305.00  1.40  74.0  1.40  0.0   
4               5.9   Flumazenil  5.9  303.00  1.30  64.0  2.20  0.0   
..              ...          ...  ...     ...   ...   ...   ...  ...   
145             0.0  Peptide 20   NaN     NaN   NaN   NaN   NaN  NaN   
146             0.0    PDK4-IN-1  NaN     NaN   NaN   NaN   NaN  NaN   
147             5.0    Berberine  NaN  336.37 -1.28  40.8 -1.28  0.0   
148             0.0   Infliximab  NaN     NaN   NaN   NaN   NaN  NaN   
149             0.0    Auranofin  NaN  678.49   NaN   NaN   NaN  NaN  